# Imports

In [1]:
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model, load_model
from tensorflow import split
import keras.losses as losses
import keras.backend as BK
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

Using TensorFlow backend.


# Read Data

In [2]:
# variablesIndex = [5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
# variablesName = ["T", "H", "H2", "O", "O2", "OH", "H2O", "HO2", "H2O2", "CO", "CO2", "CH2O",
#                  "HO2CHO", "O2CHO", "CH3O2H", "CH3O2", "CH4", "CH3", "C2H5", "C2H4", "C2H3",
#                  "CH3CHO", "C2H5OH", "O2C2H4OH"]

variablesIndex = [5, 8]
variablesName= ["T", "H"]

In [3]:
clean_data = []
for j in range(len(variablesIndex)):
  start = time.time()
  clean_data.append(pd.read_csv(
    'drive/My Drive/DNS Autoencoders/plane-xy-0.50.1.935E-03.csv',
    header=None,
    index_col=None,
    skiprows=lambda row: row < (31360 * (variablesIndex[j] - 1) + 37),
    nrows=31360,
    usecols=lambda col: col % 2 == 1,
    engine='python',
    dtype='float32'
  ))
  stop = time.time()
  print(f'Variable {variablesName[j]} Read in {round(stop-start, 2)}s')

Variable T Read in 6.63s
Variable H Read in 0.64s


# Scale + Shape Data

In [4]:
mins, maxs = [], []
for j in range(len(variablesIndex)):
    mins.append(clean_data[j].to_numpy().min())
    maxs.append(clean_data[j].to_numpy().max())
    clean_data[j] = (clean_data[j] - np.full_like(clean_data[j], mins[j])) / (np.full_like(clean_data[j], maxs[j]) - np.full_like(clean_data[j], mins[j]))

In [5]:
for j in range(len(variablesIndex)):
    clean_data[j] = np.array(clean_data[j]).reshape((560, 280))[::2]

# Create Noisy Data

In [6]:
noisy_data = list(clean_data)
for j in range(len(variablesIndex)):
    noisy_data[j] = noisy_data[j] + 0.05 * np.random.normal(loc=0.0, scale=1.0, size=noisy_data[j].shape)

# Stack Images for Training + Test

In [7]:
# stack clean and noisy training images together
clean_stacked_train = np.concatenate(clean_data).reshape((1, len(variablesIndex)*280, 280, 1))
noisy_stacked_train = np.concatenate(noisy_data).reshape((1, len(variablesIndex)*280, 280, 1))

# Condition for Mass Fractions

In [8]:
cols = [j for j in range(280)]
rows = [j for j in range(280, len(variablesIndex)*280)][::280]
rows = [[rows[i]+j for i in range(len(rows))] for j in range(280)]

def loss_conditional(y_true, y_pred):
  reshaped = BK.reshape(y_pred, (len(variablesIndex)*280, 280))
  split_images = split(reshaped, len(variablesIndex))
  sum = 0
  for j in range(280):
    for i in range(280):
      summable_points = []
      for image in split_images:
        summable_points.append(image[i, j])
      sum += BK.square(1 - BK.sum(summable_points))
  return (sum + BK.binary_crossentropy(y_true, y_pred))

losses.loss_conditional = loss_conditional

# Build Autoencoder

In [9]:
input_img = Input(shape=(len(variablesIndex)*280, 280, 1))

x = Conv2D(5, 3, padding='valid', activation='relu')(input_img)
x = MaxPooling2D(pool_size=2)(x)
encoder = Conv2D(5, 3, padding='same', activation='relu')(x)

x = Conv2DTranspose(5, 3, padding='same', activation='relu')(encoder)
x = UpSampling2D(size=2)(x)
decoder = Conv2DTranspose(5, 3, padding='valid', activation='relu')(x)

output_img = Conv2D(1, 1, padding='valid', activation='sigmoid')(decoder)

In [10]:
autoencoder = Model(input_img, output_img)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 560, 280, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 558, 278, 5)       50        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 279, 139, 5)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 279, 139, 5)       230       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 279, 139, 5)       230       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 558, 278, 5)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 560, 280, 5)       230 

# Fit Autoencoder + Load Best Model

In [ ]:
autoencoder.compile('adam', loss=loss_conditional)
autoencoder.fit(
    noisy_stacked_train, 
    clean_stacked_train,
    batch_size=None,
    epochs=50,
    callbacks=[
      ModelCheckpoint('drive/My Drive/DNS Autoencoders/bestModel_1.935E-03', monitor='loss', save_best_only=True), 
      EarlyStopping(monitor='loss', patience=3)
    ]
)

In [ ]:
best_autoencoder = load_model('drive/My Drive/DNS Autoencoders/bestModel_1.935E-03')

# Predict + Inversely Scale the Data

In [ ]:
predicted_train = np.split(best_autoencoder.predict_on_batch(noisy_stacked_train).reshape((len(variablesIndex)*280, 280)), len(variablesIndex))

In [ ]:
for j in range(len(variablesIndex)):
  predicted_train[j] = predicted_train[j]*(np.full_like(predicted_train[j], maxs[j]) - np.full_like(predicted_train[j], mins[j])) + np.full_like(predicted_train[j], mins[j])
  clean_data[j] =clean_data[j]*(np.full_like(clean_data[j], maxs[j]) - np.full_like(clean_data[j], mins[j])) + np.full_like(clean_data[j], mins[j])
  noisy_data[j] =noisy_data[j]*(np.full_like(noisy_data[j], maxs[j]) - np.full_like(noisy_data[j], mins[j])) + np.full_like(noisy_data[j], mins[j])

# Visualize on Training Set

In [ ]:
for j in range(len(variablesIndex)):
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))
  fig.suptitle(f'Variable = {variablesName[j]}', y=0.8)
  
  ax1.set_title("Clean")
  ax1.imshow(clean_data[j])
  
  ax2.set_title("Noisy")
  ax2.imshow(noisy_data[j])

  ax3.set_title("Predicted")
  ax3.imshow(predicted_train[j])

  plt.tight_layout()
  plt.savefig(f'drive/My Drive/DNS Autoencoders/1.935E-03 Results/{variablesName[j]}.tiff')

  plt.show()